# Evaluate model on Binding Affinity test set

Code to compute evaluate models on test set. It generates the following files
for each evaluated model:
- test_results.tsv: File containing labels and predictions of the model
- test_metrics.tsv: File with global metrics:
    - Accuracy
    - Precision
    - Recall
    - Average prob of true negative labels (non-binders)
    - Average prob of true positive labels (binders)
    - AUROC

In [1]:
import os
import sys
import glob
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, precision_recall_curve, auc, precision_score, recall_score, confusion_matrix

from nimbus.predictors import pHLABindingPredictor, pHLAPseudoseqBindingPredictor
from nimbus.data_processing import pHLADataset, pHLAPseudoseqDataset
from nimbus.utils import LoggerFactory

logger = LoggerFactory.get_logger('explore_pHLApredictor_nb', 'INFO')

In [2]:
DATA_DIR = '../data'
RAW_DATA = os.path.join(DATA_DIR, 'raw')
PROCESSED_DATA = os.path.join(DATA_DIR, 'processed')
HLA_FP_DIR = os.path.join(PROCESSED_DATA, 'hla_fingerprints')
hla_fp_36_data_file = os.path.join(HLA_FP_DIR, 'hla_index_netMHCpan_pseudoseq_res_representation.csv')
hla_fp_36_file = os.path.join(HLA_FP_DIR, 'hla_fingerprint_netMHCpan_pseudoseq_res_representation.npy')
hla_fp_400_data_file = os.path.join(HLA_FP_DIR, 'hla_af_patch_info_patch_r18_pt400.csv')
hla_fp_400_file = os.path.join(HLA_FP_DIR, 'hla_af_patch_emb_patch_r18_pt400.npy')
hla_pseudoseq_file = os.path.join(RAW_DATA, 'pHLA_binding', 'NetMHCpan_train', 'MHC_pseudo_fixed.dat')
test_netmhcpan_data_file = os.path.join(PROCESSED_DATA, 'pHLA_binding', 'NetMHCpan_dataset', 'test_set_peptides_data_MaxLenPep15_hla_ABC.csv.gz')


CHECKPOINTS_DIR = '../checkpoints/csv_logger'
v_num = 0  # Version number
experiments_dict = {
    # 'pHLA_balance': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_balance', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 400,
    #     'hla_representation_type': 'surface_fp',
    # },
    # 'pHLA_balance_hla_pseudoseq': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_balance_hla_pseudoseq', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 36,
    #     'hla_representation_type': 'surface_fp',
    # },
    # 'pHLA_imbalance': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_imbalance', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 400,
    #     'hla_representation_type': 'surface_fp',
    # },
    # 'pHLA_imbalance_hla_pseudoseq': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_imbalance_hla_pseudoseq', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 36,
    #     'hla_representation_type': 'surface_fp',
    # },
    # 'pHLA_balance_FILIP128': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_balance_FILIP128', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 400,
    #     'hla_representation_type': 'surface_fp',
    # },
    # 'pHLA_imbalance_hla_pseudoseq_ManSplits0123_4': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_imbalance_hla_pseudoseq_ManSplits0123_4', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 36,
    #     'hla_representation_type': 'surface_fp',
    # },
    # 'pHLA_imbalance_hla_pseudoseq_ManSplits0124_3': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_imbalance_hla_pseudoseq_ManSplits0124_3', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 36,
    #     'hla_representation_type': 'surface_fp',
    # },
    # 'pHLA_imbalance_hla_pseudoseq_ManSplits0134_2': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_imbalance_hla_pseudoseq_ManSplits0134_2', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 36,
    #     'hla_representation_type': 'surface_fp',
    # },
    # 'pHLA_imbalance_hla_pseudoseq_ManSplits0234_1': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_imbalance_hla_pseudoseq_ManSplits0234_1', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 36,
    #     'hla_representation_type': 'surface_fp',
    # },
    # 'pHLA_imbalance_hla_pseudoseq_ManSplits1234_0': {
    #     'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pHLA_imbalance_hla_pseudoseq_ManSplits1234_0', f'version_{v_num}', 'checkpoints','ep*'))[0],
    #     'hla_fp_size': 36,
    #     'hla_representation_type': 'surface_fp',
    # },
    'pseudoseq_pHLA_imbalance_ManSplits0123_4': {
        'model_checkpoint': glob.glob(os.path.join(CHECKPOINTS_DIR, 'pseudoseq_pHLA_imbalance_ManSplits0123_4', f'version_{v_num}', 'checkpoints','ep*'))[0],
        'hla_fp_size': 0,
        'hla_representation_type': 'pseudoseq',
    },
}

In [3]:
# For debugging
test_netmhcpan_data = pd.read_csv(test_netmhcpan_data_file)
# pick 500 random samples
test_netmhcpan_data = test_netmhcpan_data.sample(500, replace=False, random_state=42)
# show num 1 labels
test_netmhcpan_data[test_netmhcpan_data['label'] == 1]

peptide  is_mono_allelic  hla_allele  label
11222981  TLDYKPLSV             True  HLA-A02-01      1

In [4]:
# test_netmhcpan_data = pd.read_csv(test_netmhcpan_data_file)
hla_fp_36_emb = np.load(hla_fp_36_file)
hla_fp_400_emb = np.load(hla_fp_400_file)
hla_fp_36_data = pd.read_csv(hla_fp_36_data_file, index_col=1, names=['index'], header=0).to_dict()['index']
hla_fp_400_data = pd.read_csv(hla_fp_400_data_file, index_col=1, names=['index'], header=0).to_dict()['index']
hla_fp_36_dict = {hla: torch.Tensor(hla_fp_36_emb[idx]) for hla, idx in hla_fp_36_data.items()}
hla_fp_400_dict = {hla: torch.Tensor(hla_fp_400_emb[idx]) for hla, idx in hla_fp_400_data.items()}
hla_pseudoseq_dict = pd.read_csv(hla_pseudoseq_file, sep='\s+', names=['pseudoseq'], header=None).to_dict()['pseudoseq']


In [ ]:
test_dataset_pseudoseq_surf = pHLADataset(
    peptide_seq_arr=test_netmhcpan_data['peptide'].values,
    hla_names_arr=test_netmhcpan_data['hla_allele'].values, 
    hla_fp_dict=hla_fp_36_dict,
    labels=test_netmhcpan_data['label'].values
)

In [ ]:
test_dataset_patch_surf = pHLADataset(
    peptide_seq_arr=test_netmhcpan_data['peptide'].values,
    hla_names_arr=test_netmhcpan_data['hla_allele'].values, 
    hla_fp_dict=hla_fp_400_dict,
    labels=test_netmhcpan_data['label'].values
)

In [5]:
test_dataset_pseudoseq = pHLAPseudoseqDataset(
    peptide_seq_arr=test_netmhcpan_data['peptide'].values,
    hla_names_arr=test_netmhcpan_data['hla_allele'].values,
    hla_pseudoseq_dict=hla_pseudoseq_dict,
    labels=test_netmhcpan_data['label'].values
)

In [6]:
def load_pretrained_model(checkpoint_file, hla_representation_type='surface_fp'):
    if hla_representation_type == 'surface_fp':
        logger.info(f"Loading pHLABindingPredictor pretrained model {checkpoint_file}")
        model = pHLABindingPredictor.load_from_checkpoint(checkpoint_file)
    elif hla_representation_type == 'pseudoseq':
        logger.info(f"Loading pHLAPseudoseqBindingPredictor pretrained model {checkpoint_file}")
        model = pHLAPseudoseqBindingPredictor.load_from_checkpoint(checkpoint_file)
    else:
        logger.error(f"Unknown hla_representation_type {hla_representation_type}. "
                     f"Expected 'surface_fp' or 'pseudoseq'")
        sys.exit(1)
    return model


In [7]:
#for exp_name in experiments_dict.keys():
exp_name = 'pseudoseq_pHLA_imbalance_ManSplits0123_4'
hla_representation_type = experiments_dict[exp_name]['hla_representation_type']
model = load_pretrained_model(experiments_dict[exp_name]['model_checkpoint'], hla_representation_type=hla_representation_type)

if hla_representation_type == 'surface_fp':
    if 400 == experiments_dict[exp_name]['hla_fp_size']:
        logger.info(f"Using 400 dimensional HLA fingerprints")
        test_dataset = test_dataset_patch_surf
    elif 36 == experiments_dict[exp_name]['hla_fp_size']:
        logger.info(f"Using 36 dimensional HLA fingerprints")
        test_dataset = test_dataset_pseudoseq_surf
elif hla_representation_type == 'pseudoseq':
    logger.info(f"Using HLA pseudosequences")
    test_dataset = test_dataset_pseudoseq
else:
    logger.error(f"Unknown hla_representation_type {hla_representation_type} for {exp_name}")
    sys.exit(1)

model.eval()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
preds = []
labels = []
for i, batch in enumerate(test_loader):
    p, h, l = batch
    reps = model(p, h) # Filip Representation
    logits = model.linear_to_logits(reps)
    logits = model.to_pred(logits)
    pred = torch.sigmoid(logits).detach().cpu().numpy().tolist()
    preds.extend(pred)
    l = l.detach().cpu().numpy().tolist()
    labels.extend(l)
    
# save results to file in checkpoint folder
results = pd.DataFrame({'labels': labels, 'preds': preds})
results['labels'] = results['labels'].astype(int)
results_file = os.path.join(os.path.dirname(experiments_dict[exp_name]['model_checkpoint']), '..', 'test_results.tsv')
results.to_csv(results_file, index=False, sep='\t')
logger.info(f"Predicted test results saved to {results_file}")

metrics = {}
# compute accuracy
preds = np.array(preds)
labels = np.array(labels)
preds_05 = preds > 0.5
acc = np.mean(preds_05 == labels)
metrics['accuracy'] = acc
# compute precision: TP / (TP + FP)
precision = precision_score(labels, preds_05)
metrics['precision'] = precision
# compute recall: TP / (TP + FN)
recall = recall_score(labels, preds_05)
metrics['recall'] = recall
# compute auroc
fpr, tpr, _ = roc_curve(labels, preds)
auroc_score = auc(fpr, tpr)
metrics['auroc'] = auroc_score
# get pred value for negative class
neg_prob = np.mean(preds[labels == 0])
metrics['neg_prob'] = neg_prob
# get pred value for positive class
pos_prob = np.mean(preds[labels == 1])
metrics['pos_prob'] = pos_prob
# save metrics to file in checkpoint folder
metrics_file = os.path.join(os.path.dirname(experiments_dict[exp_name]['model_checkpoint']), '..', 'test_metrics.tsv')
pd.DataFrame(metrics, index=[0]).to_csv(metrics_file, index=False, sep='\t')
logger.info(f"Metrics saved to {metrics_file}")

3280680746 - INFO: Loading pHLAPseudoseqBindingPredictor pretrained model ../checkpoints/csv_logger/pseudoseq_pHLA_imbalance_ManSplits0123_4/version_0/checkpoints/epoch07-val_loss0.26-val_acc0.89.ckpt
3785421274 - INFO: Using HLA pseudosequences
3785421274 - INFO: Predicted test results saved to ../checkpoints/csv_logger/pseudoseq_pHLA_imbalance_ManSplits0123_4/version_0/checkpoints/../test_results.tsv
3785421274 - INFO: Metrics saved to ../checkpoints/csv_logger/pseudoseq_pHLA_imbalance_ManSplits0123_4/version_0/checkpoints/../test_metrics.tsv


# Analysis on precumputed results

## ROC-AUC curve
Plot ROC-AUC curve for all the models

In [3]:
for exp_name in experiments_dict.keys():
    results_file = os.path.join(os.path.dirname(experiments_dict[exp_name]['model_checkpoint']), '..', 'test_results.tsv')
    try:
        results = pd.read_csv(results_file, sep='\t')
    except FileNotFoundError:
        logger.error(f"File {results_file} not found")
        continue
    fpr, tpr, _ = roc_curve(results['labels'], results['preds'])
    auroc_score = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{exp_name} (area = {auroc_score:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


## Plot Precision-Recall curve

In [4]:
for exp_name in experiments_dict.keys():
    results_file = os.path.join(os.path.dirname(experiments_dict[exp_name]['model_checkpoint']), '..', 'test_results.tsv')
    try:
        results = pd.read_csv(results_file, sep='\t')
    except FileNotFoundError:
        logger.error(f"File {results_file} not found")
        continue
    precision, recall, _ = precision_recall_curve(results['labels'], results['preds'])
    # compute area under the curve
    pr_auc = auc(recall, precision)
    plt.plot(recall, precision, label=f'{exp_name} (area = {auroc_score:.2f})')
    
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend(loc="lower right")
plt.show()

## Compare metrics across models

In [5]:
metrics = {}
for exp_name in experiments_dict.keys():
    metrics_file = os.path.join(os.path.dirname(experiments_dict[exp_name]['model_checkpoint']), '..', 'test_metrics.tsv')
    try:
        metrics[exp_name] = pd.read_csv(metrics_file, sep='\t')
    except FileNotFoundError:
        logger.error(f"File {metrics_file} not found")
        continue
# Make metrics df keeping exp_name in column
metrics_df = pd.concat(metrics).reset_index(drop=False)
metrics_df = metrics_df.rename(columns={'level_0': 'exp_name'})
metrics_df = metrics_df.drop(columns='level_1')
metrics_df

exp_name  accuracy  precision  \
0                                  pHLA_balance  0.958431   0.002369   
1                    pHLA_balance_hla_pseudoseq  0.948167   0.002321   
2                                pHLA_imbalance  0.988091   0.003996   
3                  pHLA_imbalance_hla_pseudoseq  0.979942   0.005279   
4                         pHLA_balance_FILIP128  0.953159   0.001919   
5  pHLA_imbalance_hla_pseudoseq_ManSplits0123_4  0.978736   0.004918   
6  pHLA_imbalance_hla_pseudoseq_ManSplits0124_3  0.978803   0.004953   
7  pHLA_imbalance_hla_pseudoseq_ManSplits0134_2  0.976078   0.004355   
8  pHLA_imbalance_hla_pseudoseq_ManSplits0234_1  0.978686   0.004953   
9  pHLA_imbalance_hla_pseudoseq_ManSplits1234_0  0.980205   0.005298   

     recall     auroc  neg_prob  pos_prob  
0  0.699398  0.933253  0.061786  0.675039  
1  0.854819  0.967194  0.071270  0.814720  
2  0.336145  0.865760  0.025579  0.343191  
3  0.753614  0.972073  0.036209  0.699198  
4  0.637952  0.927730  0.071422  0.613067  
5  0.743976  0.970676  0.040048  0.684277  
6  0.746988  0.973378  0.036485  0.693424  
7  0.740964  0.967713  0.040145  0.693842  
8  0.751205  0.972725  0.034782  0.706061  
9  0.746386  0.974686  0.033974  0.700414

## Confusion matrix

In [6]:
# Make a plot with all confusion matrices. The confusion matrix is computed with a threshold of 0.5. The title is the experiment name.

n_plots = len(experiments_dict.keys())
n_cols = 3
n_rows = int(np.ceil(n_plots / n_cols))
fig, axs = plt.subplots(n_rows, n_cols, figsize=(15, 15))

for i, exp_name in enumerate(experiments_dict.keys()):
    results_file = os.path.join(os.path.dirname(experiments_dict[exp_name]['model_checkpoint']), '..', 'test_results.tsv')
    try:
        results = pd.read_csv(results_file, sep='\t')
    except FileNotFoundError:
        logger.error(f"File {results_file} not found")
        continue
    conf_matrix = confusion_matrix(results['labels'], results['preds'] > 0.5)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=axs[i//n_cols, i%n_cols])
    axs[i//n_cols, i%n_cols].set_title(exp_name)
    axs[i//n_cols, i%n_cols].set_xlabel('Predicted label')
    axs[i//n_cols, i%n_cols].set_ylabel('True label')
plt.tight_layout()
plt.show()

## Check false positives and false negatives

Get information about which hla alleles are more likely to be false positives and false negatives for each model when using the threshold of 0.5.

In [9]:
test_netmhcpan_data = pd.read_csv(test_netmhcpan_data_file)

In [8]:
training_ba_data_file = os.path.join(PROCESSED_DATA, 'pHLA_binding', 'NetMHCpan_dataset', 'train_binding_affinity_peptides_data_MaxLenPep15_hla_ABC_with_BalancedSplits.csv')
training_ba_data = pd.read_csv(training_ba_data_file)
# count num of positive and negative labels for each hla allele
training_ba_data['label'] = training_ba_data['label'].astype(int)
hla_allele_counts = training_ba_data.groupby('hla_allele')['label'].value_counts().unstack().fillna(0)
hla_allele_counts['train_total'] = hla_allele_counts.sum(axis=1)
hla_allele_counts = hla_allele_counts.sort_values(by='train_total', ascending=False)
hla_allele_counts = hla_allele_counts.astype(int)
hla_allele_counts = hla_allele_counts.rename(columns={0: 'train_NB', 1: 'train_B'})
# compute ratio of positive labels
hla_allele_counts['train_ratio_pos_per_total'] = hla_allele_counts['train_B'] / hla_allele_counts['train_total']
hla_allele_counts

label       train_NB  train_B  train_total  train_ratio_pos_per_total
hla_allele                                                           
HLA-A02-01      8139     4814        12953                   0.371651
HLA-A03-01      5401     2180         7581                   0.287561
HLA-A11-01      4352     2226         6578                   0.338401
HLA-A02-03      3999     2424         6423                   0.377394
HLA-A31-01      4190     1616         5806                   0.278333
...              ...      ...          ...                        ...
HLA-B15-10       104        0          104                   0.000000
HLA-C08-03       103        0          103                   0.000000
HLA-B27-10       102        0          102                   0.000000
HLA-B13-02       100        1          101                   0.009901
HLA-B55-01       100        1          101                   0.009901

[109 rows x 4 columns]

In [10]:
for exp_name in experiments_dict.keys():
    results_file = os.path.join(os.path.dirname(experiments_dict[exp_name]['model_checkpoint']), '..', 'test_results.tsv')
    try:
        results = pd.read_csv(results_file, sep='\t')
    except FileNotFoundError:
        logger.error(f"File {results_file} not found")
        continue
    results['preds_05'] = results['preds'] > 0.5
    false_positives = results[(results['labels'] == 0) & (results['preds_05'] == 1)]
    fp_index = false_positives.index
    fp_data = test_netmhcpan_data.iloc[fp_index]
    fp_hla_alleles = fp_data['hla_allele'].value_counts()
    # add columns with training counts per allele
    fp_hla_alleles = pd.concat([fp_hla_alleles, hla_allele_counts], axis=1, join='inner')#.sort_values(by='total', ascending=False)
    false_negatives = results[(results['labels'] == 1) & (results['preds_05'] == 0)]
    fn_index = false_negatives.index
    fn_data = test_netmhcpan_data.iloc[fn_index]
    fn_hla_alleles = fn_data['hla_allele'].value_counts()
    # add columns with training counts per allele
    fn_hla_alleles = pd.concat([fn_hla_alleles, hla_allele_counts], axis=1, join='inner')#.sort_values(by='total', ascending=False)
    
    logger.info(f"False positives (non-binders classified as binders) for {exp_name} were {len(fp_data)}:")
    logger.info(fp_hla_alleles)
    logger.info(f"False negatives (binders classified as non-binders) for {exp_name} were {len(fn_data)}:")
    logger.info(fn_hla_alleles)
    break

3564553374 - INFO: False positives (non-binders classified as binders) for pHLA_balance were 488863:
3564553374 - INFO:              count  train_NB  train_B  train_total  train_ratio_pos_per_total
hla_allele                                                                   
HLA-A02-01  180043      8139     4814        12953                   0.371651
HLA-B07-02   68871      3548     1401         4949                   0.283087
HLA-A11-01   34988      4352     2226         6578                   0.338401
HLA-B08-01   23028      2841      803         3644                   0.220362
HLA-B58-01   21092      2718      763         3481                   0.219190
HLA-A03-01   18421      5401     2180         7581                   0.287561
HLA-A01-01   14874      4249      734         4983                   0.147301
HLA-A24-02   14387      2710      753         3463                   0.217442
HLA-B57-01   13052      2607      493         3100                   0.159032
HLA-B35-01   12711    

In [11]:
from sklearn.metrics import classification_report
print(classification_report(results['labels'], results['preds']>0.5))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98  11770570
           1       0.00      0.70      0.00      1660

    accuracy                           0.96  11772230
   macro avg       0.50      0.83      0.49  11772230
weighted avg       1.00      0.96      0.98  11772230

